In [1]:
import drdf
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def load_drdf(fname):
  reader = drdf.DRDF()
  reader.read(fname)
  events = []
  for run in reader.runs:
    for event in reader.runs[run]:
      hits_map = dict()
      for cam, img in reader.runs[run][event].items():
        amplitude = img.pixels[:, :, 0] 
        time = img.pixels[:, :, 1]
        hits_map[cam] = amplitude
      events.append((event, hits_map))
  return events

file = load_drdf("/Users/giacomosantoni/Desktop/TESI/Progetto_ML/blindcams/response.drdf")

In [3]:
def SumPhotonsAllCams():
    all_cam_ev = []
    for i in range(len(file)):
        all_cam_list = []
        sum_cam_all = [0.]#definisco questa lista perchè mi serve dopo
        event_i = file[i][1]# così sto considerando file, nell'evento i esimo, la parte di hits_map (che  un dict)
        for cam in event_i:
            sum_cam_all = sum(map(sum, event_i[cam]))#nella camera x faccio la somma dei fotoni su ogni pixel
            all_cam_list.append(sum_cam_all)#ogni somma di fotoni relativa a una camera la aggiungo in una lista di tutte le camere in ogni evento
        all_cam_ev.append(all_cam_list)
    return all_cam_ev


In [4]:
def CamList():
    cam_list = []
    event_i = file[151][1]
    for cam in event_i:
        if np.any(event_i[cam] != 0.):
            cam_list.append(cam)
    return cam_list

#all_cam_list = SumPhotonsCam1event()
cam_list = CamList()
print(len(cam_list))

54


SCALING

In [5]:
from sklearn.preprocessing import RobustScaler

def ScalingData():
    sum_photons_all_cams_all_ev = np.array(SumPhotonsAllCams())
    sum_photons_not_0_list = []
    for i in range(len(sum_photons_all_cams_all_ev)):
        if not(np.any(sum_photons_all_cams_all_ev[i])):
            continue
        sum_photons_not_0_list.append(sum_photons_all_cams_all_ev[i])
    sum_photons_not_0 = np.array(sum_photons_not_0_list)
    sum_photons_not_0_tr = np.transpose(sum_photons_not_0)

    transformer = RobustScaler().fit(sum_photons_not_0_tr)
    scaled_data = transformer.transform(sum_photons_not_0_tr)
    scaled_data_tr = np.transpose(scaled_data)

    n=0
    scaled_sum_photons = []
    for i in range(len(sum_photons_all_cams_all_ev)):
        if np.any(sum_photons_all_cams_all_ev[i]) != 0:
            sum_photons_all_cams_all_ev[i]=scaled_data_tr[n]
            n += 1
    scaled_sum_photons = sum_photons_all_cams_all_ev
    return scaled_sum_photons

In [6]:
scaled_sum_photons = ScalingData()
print(len(scaled_sum_photons[0]))

54


ROOT

In [9]:
import ROOT as root

#sensor1.root è il file; ogni camera è un TTree
input_file = root.TFile.Open("/Users/giacomosantoni/Desktop/TESI/Progetto_ML/blindcams/sensors.root", "READ")
tree = input_file.Get("CAM_NB_X1")
entries = tree.GetEntries()

cam_list = CamList()

nr_photons_list_all_cams_list = []
for cam in cam_list:
    nr_photons_list = []
    tree = input_file.Get(cam)
    for i in range(entries):
        n = tree.GetEntry(i)
        inner_photons = tree.innerPhotons
        #if inner_photons: 
        nr_photons_list.append(inner_photons)
    #print(nr_photons_list)
    nr_photons_list_all_cams_list.append(nr_photons_list)

nr_photons_list_all_cams = np.array(nr_photons_list_all_cams_list)#ho creato una matrice 10.000 colonne (nr eventi) x 76 righe (nr camere). I numeri che vediamo sono i numeri di fotoni che arrivano alla camera


Welcome to JupyROOT 6.28/04


In [10]:
#print(len(nr_photons_list_all_cams[0]))

nr_photons_list_all_cams_tr = nr_photons_list_all_cams.transpose()

CNN MODEL

In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models


from sklearn.model_selection import train_test_split

In [69]:
train_ds, val_ds, train_labels, val_labels = train_test_split(scaled_sum_photons, nr_photons_list_all_cams_tr, train_size=0.8, random_state=42)
#train_labels = ev_cam_state

# print("TRAIN: ", list(train_ds))
# print("VALIDATION: ", list(val_ds))

#print(len(train_ds[0]),len(val_ds),len(train_labels),len(val_labels))

print(train_ds.shape)
print(val_ds.shape)
print(train_labels.shape)
print(val_labels.shape)


(800, 54)
(200, 54)
(800, 54)
(200, 54)
